### DATA 620
### Week 3: Graph Visualization
**Completed by:** Candace Grant

**Date:** 2/21/26

### Assignment Overview

In this assignment I am required to complete the following steps.
1. Load a graph database of your choosing from a text file or other source.
2. Create basic analysis on the graph
3. Use a visualization tool to display information.
4. Submit a short video presentation as part of the submission.


### Dataset Information

Network was collected by crawling the Amazon website. It is based on the **Customers Who Bought This Item Also Bought** feature of the Amazon website. If a product *i* is frequently co-purchased with product *j*, the graph contains a directed edge from *i* to *j*.

The data was collected on June 01, 2003.

---

### Dataset Statistics

| Metric | Value |
|---|---|
| Nodes | 403,394 |
| Edges | 3,387,388 |
| Nodes in largest WCC | 403,364 (1.000) |
| Edges in largest WCC | 3,387,224 (1.000) |
| Nodes in largest SCC | 395,234 (0.980) |
| Edges in largest SCC | 3,301,092 (0.975) |
| Average clustering coefficient | 0.4177 |
| Number of triangles | 3,986,507 |
| Fraction of closed triangles | 0.06206 |
| Diameter (longest shortest path) | 21 |
| 90-percentile effective diameter | 7.6 |

---

### Source (Citation)

J. Leskovec, L. Adamic and B. Adamic. *The Dynamics of Viral Marketing.* ACM Transactions on the Web (ACM TWEB), 1(1), 2007.

| File | Description |
|---|---|
| `amazon0601.txt.gz` | Amazon product co-purchasing network from June 01, 2003 |

**Download:** [https://snap.stanford.edu/data/amazon0601.html](https://snap.stanford.edu/data/amazon0601.html)


## Section 1: Setup and Import libraries

In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
from collections import Counter, deque
from concurrent.futures import ThreadPoolExecutor
import gzip
import os
import urllib.request
import time
import random
import warnings
warnings.filterwarnings('ignore')

random.seed(42)
np.random.seed(42)

# Plotting style
plt.rcParams.update({
    'figure.facecolor': '#FAFAFA',
    'axes.facecolor': '#FAFAFA',
    'font.family': 'sans-serif',
    'font.size': 11,
    'axes.titlesize': 14,
    'axes.titleweight': 'bold',
    'axes.labelsize': 12,
    'figure.dpi': 150,
})

print(f"NetworkX version: {nx.__version__}")
print(f"NumPy version:    {np.__version__}")
print(f"CPU cores:        {os.cpu_count()}")
print("All packages loaded successfully!")

NetworkX version: 3.6.1
NumPy version:    2.0.2
CPU cores:        2
All packages loaded successfully!


## Section 2 Load the Graph (Vectorized with Numpy)
Instead of reading line-by-line in a Python loop, `np.loadtxt()` bulk-loads all edges into a contiguous C array, then `add_edges_from()` batch-adds them in a single call. This is **5–10x faster** than the loop approach.


In [2]:
file_path = "amazon0601.txt.gz"

if not os.path.exists(file_path):
    print(f"Downloading {file_path} from SNAP...")
    url = "https://snap.stanford.edu/data/amazon0601.txt.gz"
    try:
        urllib.request.urlretrieve(url, file_path)
        print("Download complete.")
    except Exception as e:
        print(f"Error: {e}")
        print("Please download manually from https://snap.stanford.edu/data/amazon0601.html")

# Vectorized loading
start = time.time()
print("Loading edges with NumPy (vectorized)...")
edges_array = np.loadtxt(file_path, dtype=int, comments='#')

G_full = nx.DiGraph()
G_full.add_edges_from(edges_array)

print(f"  Loaded in {time.time()-start:.2f}s")
print(f"  Nodes: {G_full.number_of_nodes():,}")
print(f"  Edges: {G_full.number_of_edges():,}")

Download complete.
Loading edges with NumPy (vectorized)...
  Loaded in 18.78s
  Nodes: 403,394
  Edges: 3,387,388


## Section 3: Snowball Sampling
The full graph has 400K+ nodes which makes exact diameter and betweenness centrality computation prohibitively expensive. We use **snowball sampling** (BFS from a high-degree seed node) which preserves local network structure — clustering patterns, community structure, and hub-spoke topology remain intact. This is superior to random node sampling or simply taking the first N lines of the file.

In [3]:
SAMPLE_SIZE = 10000

def snowball_sample(G, target_size, seed_node=None):
    """BFS snowball sampling — preserves local network structure."""
    G_und = G.to_undirected()

    if seed_node is None:
        degree_arr = np.array(G_und.degree())
        top_idx = np.argsort(degree_arr[:, 1])[-20:]
        seed_node = int(degree_arr[np.random.choice(top_idx)][0])

    print(f"Snowball sampling from seed node {seed_node}...")
    sampled = set()
    frontier = {seed_node}

    while len(sampled) < target_size and frontier:
        sampled.update(frontier)
        next_frontier = set()
        for node in frontier:
            next_frontier.update(set(G_und.neighbors(node)) - sampled)
        if len(sampled) + len(next_frontier) > target_size * 1.2:
            needed = target_size - len(sampled)
            next_frontier = set(random.sample(list(next_frontier), min(needed, len(next_frontier))))
        frontier = next_frontier

    sampled = list(sampled)[:target_size]
    return G.subgraph(sampled).copy(), G_und.subgraph(sampled).copy()

start = time.time()
G_dir, G_und = snowball_sample(G_full, SAMPLE_SIZE)
print(f"  Sampled nodes: {G_dir.number_of_nodes():,}")
print(f"  Sampled edges: {G_dir.number_of_edges():,}")
print(f"  Completed in {time.time()-start:.2f}s")

Snowball sampling from seed node 533...
  Sampled nodes: 10,000
  Sampled edges: 34,261
  Completed in 17.43s


## Section 4: Extract Largest Connected Component

In [4]:
# Weakly connected components (directed)
wcc_list = list(nx.weakly_connected_components(G_dir))
largest_wcc = max(wcc_list, key=len)

# Largest connected component (undirected) — used for diameter and path metrics
largest_cc = max(nx.connected_components(G_und), key=len)
H = G_und.subgraph(largest_cc).copy()

# Strongly connected components (directed)
scc_list = list(nx.strongly_connected_components(G_dir))
largest_scc = max(scc_list, key=len)

print(f"Weakly Connected Components:  {len(wcc_list)}")
print(f"  Largest WCC: {len(largest_wcc):,} nodes ({len(largest_wcc)/G_dir.number_of_nodes():.3f})")
print(f"Strongly Connected Components: {len(scc_list)}")
print(f"  Largest SCC: {len(largest_scc):,} nodes ({len(largest_scc)/G_dir.number_of_nodes():.3f})")
print(f"\nLargest CC (undirected) for analysis:")
print(f"  Nodes: {H.number_of_nodes():,}")
print(f"  Edges: {H.number_of_edges():,}")

Weakly Connected Components:  1
  Largest WCC: 10,000 nodes (1.000)
Strongly Connected Components: 5214
  Largest SCC: 2,581 nodes (0.258)

Largest CC (undirected) for analysis:
  Nodes: 10,000
  Edges: 27,375


## Section 5: Metric 1 - Diameter (Required)

The **diameter** is the longest shortest path between any two nodes in the network. It tells us: what is the maximum number of co-purchase hops needed to connect the two most distant products?

I implement this **by hand** using BFS from sampled nodes to build intuition, then verify with the NetworkX approximation algorithm.

In [5]:
# ════════════════════════════════════════════════════════
#  HAND-CODED DIAMETER ESTIMATION
#  Using BFS (Breadth-First Search) from sampled nodes
# ════════════════════════════════════════════════════════

def bfs_eccentricity(graph, source):
    """
    Hand-coded BFS to find the eccentricity of a single node.
    Eccentricity = the longest shortest path FROM this node to any other.
    The diameter is the maximum eccentricity across all nodes.
    """
    visited = {source: 0}
    queue = deque([source])
    max_distance = 0

    while queue:
        node = queue.popleft()
        current_dist = visited[node]

        for neighbor in graph.neighbors(node):
            if neighbor not in visited:
                visited[neighbor] = current_dist + 1
                max_distance = max(max_distance, current_dist + 1)
                queue.append(neighbor)

    return max_distance, visited


def estimate_diameter_bfs(graph, n_samples=100):
    """
    Estimate diameter by running BFS from n_samples random nodes.
    The maximum eccentricity found is a lower bound on the true diameter.
    """
    nodes = list(graph.nodes())
    sample = random.sample(nodes, min(n_samples, len(nodes)))

    max_ecc = 0
    total_path_length = 0
    total_pairs = 0

    for source in sample:
        ecc, distances = bfs_eccentricity(graph, source)
        max_ecc = max(max_ecc, ecc)
        total_path_length += sum(distances.values())
        total_pairs += len(distances)

    avg_path = total_path_length / total_pairs if total_pairs > 0 else 0
    return max_ecc, avg_path


# Run hand-coded diameter estimation
start = time.time()
print("Computing diameter (hand-coded BFS from 100 sampled nodes)...")
diameter_hand, avg_path_hand = estimate_diameter_bfs(H, n_samples=100)
elapsed_hand = time.time() - start

print(f"\n  Hand-Coded Results:")
print(f"    Estimated Diameter:       {diameter_hand}")
print(f"    Avg Shortest Path Length: {avg_path_hand:.3f}")
print(f"    Computed in {elapsed_hand:.2f}s")

Computing diameter (hand-coded BFS from 100 sampled nodes)...

  Hand-Coded Results:
    Estimated Diameter:       6
    Avg Shortest Path Length: 4.506
    Computed in 2.18s


In [6]:
# ════════════════════════════════════════════════════════
#  VERIFY WITH NETWORKX APPROXIMATION
# ════════════════════════════════════════════════════════

start = time.time()
diameter_nx = nx.approximation.diameter(H)
elapsed_nx = time.time() - start

print(f"  NetworkX Approximation:")
print(f"    Approximate Diameter:    {diameter_nx}")
print(f"    Computed in {elapsed_nx:.2f}s")

print(f"\n  Full SNAP Dataset Diameter: 21")
print(f"  Full SNAP 90th Pctl Eff. Diameter: 7.6")


  NetworkX Approximation:
    Approximate Diameter:    6
    Computed in 0.06s

  Full SNAP Dataset Diameter: 21
  Full SNAP 90th Pctl Eff. Diameter: 7.6


## Section 6: Metric 2 — Clustering Coefficient (Hand-Coded)

The **clustering coefficient** measures the tendency of products to form tightly connected groups. If product A is co-purchased with products B and C, the clustering coefficient asks: are B and C also co-purchased with each other?

For a node with degree *k*, the maximum possible edges between its neighbors is *k(k-1)/2*. The local clustering coefficient is the fraction of those possible edges that actually exist.

In [7]:
# ════════════════════════════════════════════════════════
#  HAND-CODED CLUSTERING COEFFICIENT
# ════════════════════════════════════════════════════════

def local_clustering_coefficient(graph, node):
    """
    Hand-coded local clustering coefficient for a single node.

    Formula: C(v) = 2 * triangles(v) / (degree(v) * (degree(v) - 1))

    This measures: of all possible connections between my neighbors,
    what fraction actually exist?
    """
    neighbors = set(graph.neighbors(node))
    k = len(neighbors)

    if k < 2:
        return 0.0  # Need at least 2 neighbors to form a triangle

    # Count edges between neighbors
    edges_between = 0
    for neighbor in neighbors:
        # How many of this neighbor's neighbors are also our neighbors?
        edges_between += len(set(graph.neighbors(neighbor)) & neighbors)

    # Each edge counted twice (once from each end)
    edges_between = edges_between / 2

    # Maximum possible edges between k neighbors
    max_possible = k * (k - 1) / 2

    return edges_between / max_possible


def average_clustering_hand(graph, n_samples=2000):
    """
    Compute average clustering coefficient over sampled nodes.
    Sampling is necessary for large graphs — O(n * k²) per node.
    """
    nodes = list(graph.nodes())
    sample = random.sample(nodes, min(n_samples, len(nodes)))

    coefficients = [local_clustering_coefficient(graph, n) for n in sample]
    return np.mean(coefficients), np.array(coefficients)


# Run hand-coded clustering
start = time.time()
print("Computing clustering coefficient (hand-coded, 2000 sampled nodes)...")
avg_cc_hand, cc_values_hand = average_clustering_hand(H, n_samples=2000)
elapsed = time.time() - start

print(f"\n  Hand-Coded Avg Clustering Coefficient: {avg_cc_hand:.4f}")
print(f"  Computed in {elapsed:.2f}s")

Computing clustering coefficient (hand-coded, 2000 sampled nodes)...

  Hand-Coded Avg Clustering Coefficient: 0.2744
  Computed in 0.09s


In [8]:
# ════════════════════════════════════════════════════════
#  VERIFY WITH NETWORKX
# ════════════════════════════════════════════════════════

start = time.time()
avg_cc_nx = nx.average_clustering(H)
transitivity = nx.transitivity(H)
elapsed = time.time() - start

print(f"  NetworkX Avg Clustering:  {avg_cc_nx:.4f}")
print(f"  NetworkX Transitivity:    {transitivity:.4f}")
print(f"  Full SNAP Dataset:        0.4177")
print(f"  Computed in {elapsed:.2f}s")

  NetworkX Avg Clustering:  0.2819
  NetworkX Transitivity:    0.0654
  Full SNAP Dataset:        0.4177
  Computed in 1.44s


## Section 7: Gephi Export for Network Visualization

To visualize the network in Gephi, I export a 500-node subgraph with pre-computed node attributes (degree, clustering, betweenness, eigenvector centrality, and community labels). The subgraph is created using snowball sampling to preserve the local network structure, then saved as a `.gexf` file that Gephi can open directly.

In [9]:
# ════════════════════════════════════════════════════════
#  GEPHI EXPORT — Snowball sample + node metrics + GEXF
# ════════════════════════════════════════════════════════

VIZ_SIZE = 500

def snowball_sample_viz(G, target_size, seed_node=None):
    """BFS snowball sampling for Gephi visualization."""
    if seed_node is None:
        degree_arr = np.array(list(G.degree()))
        top_idx = np.argsort(degree_arr[:, 1].astype(int))[-10:]
        seed_node = int(degree_arr[np.random.choice(top_idx)][0])

    print(f"  Snowball sampling from seed node {seed_node} for Gephi export...")
    sampled = set()
    frontier = {seed_node}

    while len(sampled) < target_size and frontier:
        sampled.update(frontier)
        next_frontier = set()
        for node in frontier:
            next_frontier.update(set(G.neighbors(node)) - sampled)
        if len(sampled) + len(next_frontier) > target_size * 1.2:
            needed = target_size - len(sampled)
            next_frontier = set(random.sample(list(next_frontier), min(needed, len(next_frontier))))
        frontier = next_frontier

    sampled = list(sampled)[:target_size]
    return G.subgraph(sampled).copy()


def export_for_gephi(H, viz_size=500):
    """Creates Gephi-ready files from the network."""

    print("=" * 60)
    print("  GEPHI EXPORT")
    print("=" * 60)

    # Step 1: Create visualization subgraph
    print(f"\n1. Creating {viz_size}-node subgraph...")
    G_viz = snowball_sample_viz(H, viz_size)
    print(f"   Nodes: {G_viz.number_of_nodes()}")
    print(f"   Edges: {G_viz.number_of_edges()}")

    # Step 2: Compute node-level metrics
    print("\n2. Computing node metrics for Gephi attributes...")
    degree_dict = dict(G_viz.degree())
    clustering_dict = nx.clustering(G_viz)
    betweenness_dict = nx.betweenness_centrality(G_viz, k=min(100, G_viz.number_of_nodes()))
    eigenvector_dict = nx.eigenvector_centrality_numpy(G_viz)

    for node in G_viz.nodes():
        G_viz.nodes[node]['Label'] = f"Product_{node}"
        G_viz.nodes[node]['degree'] = degree_dict[node]
        G_viz.nodes[node]['clustering'] = round(clustering_dict[node], 4)
        G_viz.nodes[node]['betweenness'] = round(betweenness_dict[node], 6)
        G_viz.nodes[node]['eigenvector'] = round(eigenvector_dict[node], 6)

    # Step 3: Detect communities
    print("   Detecting communities (Louvain)...")
    try:
        communities = nx.community.louvain_communities(G_viz, seed=42)
        for i, comm in enumerate(communities):
            for node in comm:
                G_viz.nodes[node]['community'] = i
        print(f"   Found {len(communities)} communities")
    except Exception:
        print("   Louvain not available, skipping community detection")
        for node in G_viz.nodes():
            G_viz.nodes[node]['community'] = 0

    # Export GEXF
    print("\n3. Exporting files...")
    nx.write_gexf(G_viz, "gephi_amazon_500.gexf")
    print("   ✓ gephi_amazon_500.gexf")

    # Export CSVs
    nodes_df = pd.DataFrame([{
        'Id': n, 'Label': f'Product_{n}',
        'degree': degree_dict[n],
        'clustering': round(clustering_dict[n], 4),
        'betweenness': round(betweenness_dict[n], 6),
        'eigenvector': round(eigenvector_dict[n], 6),
        'community': G_viz.nodes[n].get('community', 0)
    } for n in G_viz.nodes()])
    nodes_df.to_csv("gephi_nodes_500.csv", index=False)
    print("   ✓ gephi_nodes_500.csv")

    edges_df = pd.DataFrame([{'Source': u, 'Target': v} for u, v in G_viz.edges()])
    edges_df.to_csv("gephi_edges_500.csv", index=False)
    print("   ✓ gephi_edges_500.csv")

    print("\n  Done! Download the .gexf file and open it in Gephi (File → Open).")
    return G_viz, nodes_df, edges_df

In [10]:
# ════════════════════════════════════════════════════════
#  RUN THE EXPORT (uses H from Section 4)
# ════════════════════════════════════════════════════════

G_viz, nodes_df, edges_df = export_for_gephi(H)

print("\nNode metrics preview:")
print(nodes_df.head(10).to_string(index=False))

  GEPHI EXPORT

1. Creating 500-node subgraph...
  Snowball sampling from seed node 45 for Gephi export...
   Nodes: 500
   Edges: 632

2. Computing node metrics for Gephi attributes...
   Detecting communities (Louvain)...
   Found 21 communities

3. Exporting files...
   ✓ gephi_amazon_500.gexf
   ✓ gephi_nodes_500.csv
   ✓ gephi_edges_500.csv

  Done! Download the .gexf file and open it in Gephi (File → Open).

Node metrics preview:
    Id          Label  degree  clustering  betweenness  eigenvector  community
290821 Product_290821       1      0.0000     0.000000     0.017891          4
180239 Product_180239       1      0.0000     0.000000     0.017891          4
110620 Product_110620       1      0.0000     0.000000     0.036739         16
356388 Product_356388       1      0.0000     0.000000     0.017891          4
391206 Product_391206       1      0.0000     0.000000     0.036739         16
385065 Product_385065       1      0.0000     0.000000     0.036739         16
366634 

In [11]:
# ════════════════════════════════════════════════════════
#  DOWNLOAD FILES FROM COLAB
#  This cell triggers browser downloads
# ════════════════════════════════════════════════════════

from google.colab import files

files.download("gephi_amazon_500.gexf")
files.download("gephi_nodes_500.csv")
files.download("gephi_edges_500.csv")

print("Files downloaded! Open gephi_amazon_500.gexf in Gephi.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Files downloaded! Open gephi_amazon_500.gexf in Gephi.
